---
# Section 1.3: Triangular Systems
---

Let $L \in \mathbb{R}^{n \times n}$ be a **lower-triangular matrix** and $U \in \mathbb{R}^{n \times n}$ be an **upper-triangular matrix**. Then,

$$
L = 
\begin{bmatrix}
l_{11} \\
\vdots & \ddots \\
l_{n1} & \cdots & l_{nn}
\end{bmatrix}
\qquad \text{and} \qquad
U = 
\begin{bmatrix}
u_{11} & \cdots & u_{1n} \\
& \ddots & \vdots \\
& & u_{nn}
\end{bmatrix}.
$$

> ### Theorem:
> Let $G$ be an $n \times n$ triangular matrix. Then
> 
> $$ 
\text{$G$ is nonsingular} \quad \Longleftrightarrow \quad g_{ii} \neq 0, \quad i = 1,\ldots,n.
$$

---
### Exercise: 

Prove the above theorem.

### Proof:

Since $G$ is triangular, $\det(G) = g_{11} g_{22} \cdots g_{nn}$. Therefore, $\det(G) \ne 0$ if and only if $g_{ii} \ne 0$ for $i = 1,\ldots,n$. $\blacksquare$

---

### Exercise:
Solve the following lower-triangular system $Lx = b$ and use `Julia` to check your answer.

$$
\begin{bmatrix}
2 & 0 & 0 & 0 \\
-1 & 2 & 0 & 0 \\
3 & 1 & -1 & 0 \\
4 & 1 & -3 & 3
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4
\end{bmatrix} =
\begin{bmatrix}
2 \\ 3 \\ 2 \\ 9
\end{bmatrix}.
$$

$2 x_1 = 2 \implies x_1 = 1$

$-x_1 + 2x_2 = 3 \implies -1 + 2x_2 = 3 \implies 2x_2 = 4 \implies x_2 = 2$

$3x_1 + x_2 - x_3 = 2 \implies 3 + 2 - x_3 = 2 \implies -x_3 = -3 \implies x_3 = 3$

$4x_1 + x_2 - 3x_3 + 3x_4 = 9 \implies 4 + 2 - 9 + 3x_4 = 9 \implies 3x_4 = 12 \implies x_4 = 4$

Therefore,

$$
x =
\begin{bmatrix}
1 \\ 2 \\ 3 \\ 4
\end{bmatrix}.
$$

In [ ]:
L = [
    2 0 0 0
    -1 2 0 0
    3 1 -1 0
    4 1 -3 3.0
]
b = [2, 3, 2, 9.0]
x = L\b

In [ ]:
using LinearAlgebra

In [ ]:
istril(L)

In [ ]:
M = LowerTriangular(L)

In [ ]:
istril(M)

---

In general, the solution of a lower-triangular system $Lx = b$ is given by

$$
x_i = \left( b_i - \sum_{j=1}^{i-1} l_{ij}x_j \right) \bigg/ l_{ii}, \quad i = 1, \ldots, n.
$$

---

### Exercise:

Write a `Julia` function, `x = row_fs(L, b)`, that returns the solution $x$ of a lower-triangular system $Lx = b$ using _forward substitution_. Make sure to include a check that the matrix $L$ is nonsingular. Test your code for accuracy and efficiency.

In [ ]:
L

In [ ]:
b

In [ ]:
function row_fs(L, b)
    #if any(diag(L) .== 0)
    #    error("Error: Matrix L is singular.")
    #end
    n = length(b)
    x = copy(b)
    for i = 1:n
        for j = 1:i-1
            x[i] -= L[i,j]*x[j]
        end
        if L[i,i] == 0
            error("Error: Matrix L is singular.")
        end
        x[i] /= L[i,i]
    end
    return x
end

In [ ]:
x = row_fs(L,b)

In [ ]:
L = [
    1 0 0
    1 0 0
    2 3 1.0
]

In [ ]:
det(L)

In [ ]:
x = row_fs(L,b)

In [ ]:
n = 100
L = LowerTriangular(Float64.(rand(1:9, n, n)))
x_true = randn(n)
b = L*x_true

x = row_fs(L,b)

abs.(x - x_true)

In [ ]:
x = L\b
abs.(x - x_true)

In [ ]:
residual = b - L*x

In [ ]:
cond(L)

**Note:** This matrix $L$ has a large condition number, so solving $Lx = b$ is very sensitive to small changes in the input.

In [ ]:
A = rand(n,n)

In [ ]:
F = qr(A)
L = F.R'
cond(L)

**Note:** This lower triangular matrix $L$ has a much smaller condition number so is less sensitive to small changes in the input.

In [ ]:
b = L*x_true
x = row_fs(L, b)
abs.(x_true - x)

Our function `row_fs` appears to give accurate solutions to $Lx = b$ when $L$ is not ill-condition (having a large condition number).

In [ ]:
using BenchmarkTools

In [ ]:
n = 1000
L = LowerTriangular(rand(n,n))
b = rand(n);

In [ ]:
@btime row_fs(L,b);

In [ ]:
@btime L\b;

In [ ]:
@which L\b

---

## Column-oriented forward substitution

Notice that the algorithm above is **row-oriented**, which is good for computer languages that store matrices in row-major order (like C/C++ or Python), but is bad for column-major order languages (like Fortran, MATLAB, or Julia).

We can produce a forward-substitution algorithm that is **column-oriented** by using the concept of **block-matrix multiplication**.

---

### Exercise:

Let the lower-triangular linear system $Lx = b$ be partitioned as

$$
\begin{bmatrix}
l_{11} & 0 \\
\hat{l} & \hat{L} \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\ \hat{x}
\end{bmatrix} = 
\begin{bmatrix}
b_1 \\ \hat{b}
\end{bmatrix}
$$

where $\hat{L}$ is an $(n-1) \times (n-1)$ *lower-triangular* matrix and $\hat{x}$ and $\hat{b}$ are vectors of length $n - 1$.

1. Perform block-matrix multiplication.

2. Determine a _recursive algorithm_ for solving the linear system $Lx = b$.

3. Implement your recursive algorithm in `Julia` (name your function `col_rfs`).

4. Test your code for accuracy and efficiency.

$$
\begin{bmatrix}
l_{11} & 0 \\
\hat{l} & \hat{L} \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\ \hat{x}
\end{bmatrix} = 
\begin{bmatrix}
l_{11} x_1 + 0 \hat{x} \\
\hat{l} x_1 + \hat{L} \hat{x}
\end{bmatrix} = 
\begin{bmatrix}
l_{11} x_1 \\
x_1 \hat{l} + \hat{L} \hat{x}
\end{bmatrix}
$$

Therefore,

$$
\begin{bmatrix}
l_{11} x_1 \\
x_1 \hat{l} + \hat{L} \hat{x}
\end{bmatrix} =
\begin{bmatrix}
b_1 \\ \hat{b}
\end{bmatrix}.
$$

Thus, we have:

1. $l_{11} x_1 = b_1$,
2. $x_1 \hat{l} + \hat{L} \hat{x} = \hat{b}$.

We can solve the first equation for $x_1$:

$$
x_1 = \frac{b_1}{l_{11}}.
$$

Substituting $x_1$ into the second equation gives us:

$$
\hat{L} \hat{x} = \hat{b} - x_1 \hat{l}.
$$

---

### Exercise:

Use column-oriented forward substitution to solve the lower-triangular system.

$$
\begin{bmatrix}
2 & 0 & 0 & 0 \\
-1 & 2 & 0 & 0 \\
3 & 1 & -1 & 0 \\
4 & 1 & -3 & 3
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4
\end{bmatrix} =
\begin{bmatrix}
2 \\ 3 \\ 2 \\ 9
\end{bmatrix}
$$


First we have $x_1 = b_1/l_{11} = 2/2 = 1$. Then,

$$
\hat{b} - x_1 \hat{l} = 
\begin{bmatrix}
3 \\ 2 \\ 9
\end{bmatrix}
- 1
\begin{bmatrix}
-1 \\ 3 \\ 4
\end{bmatrix} = 
\begin{bmatrix}
4 \\ -1 \\ 5
\end{bmatrix}.
$$

Thus, we just need to solve

$$
\begin{bmatrix}
2 & 0 & 0 \\
1 & -1 & 0 \\
1 & -3 & 3
\end{bmatrix}
\begin{bmatrix}
x_2 \\ x_3 \\ x_4
\end{bmatrix} =
\begin{bmatrix}
4 \\ -1 \\ 5
\end{bmatrix}.
$$

Next we have $x_2 = 4/2 = 2$. Then, our reduced right-hand-side is

$$ 
\begin{bmatrix}
-1 \\ 5
\end{bmatrix}
- 2
\begin{bmatrix}
1 \\ 1
\end{bmatrix} = 
\begin{bmatrix}
-3 \\ 3
\end{bmatrix}.
$$

Thus, we just need to solve

$$
\begin{bmatrix}
-1 & 0 \\
-3 & 3
\end{bmatrix}
\begin{bmatrix}
x_3 \\ x_4
\end{bmatrix} =
\begin{bmatrix}
-3 \\ 3
\end{bmatrix}.
$$

Now we have $x_3 = -3/(-1) = 3$. Our new reduced right-hand-side is

$$ 
\begin{bmatrix}
3
\end{bmatrix}
- 3
\begin{bmatrix}
-3
\end{bmatrix} = 
\begin{bmatrix}
12
\end{bmatrix}.
$$

Thus, we just need to solve

$$
\begin{bmatrix}
3
\end{bmatrix}
\begin{bmatrix}
x_4
\end{bmatrix} =
\begin{bmatrix}
12
\end{bmatrix}.
$$

Thus, $x_4 = 12/3 = 4$.

---

### Exercise:

Inspired by your calculations in the previous exercise, write a non-recursive column-oriented forward-substitution code in `Julia` (name your function `col_fs`). Test your function for accuracy and efficiency.

In [ ]:
function col_fs(L, b)
    n = length(b)
    x = copy(b)
    #@show x
    for j = 1:n
        if L[j,j] == 0
            error("Error: Matrix L is singular.")
        end
        x[j] /= L[j,j]
        # Compute the reduced rhs
        for i = j+1:n
            x[i] -= L[i,j]*x[j]
        end
        #@show x
    end
    return x
end

In [ ]:
L = [
    2 0 0 0
    -1 2 0 0
    3 1 -1 0
    4 1 -3 3.0
]
b = [2, 3, 2, 9.0]

x = col_fs(L, b)

In [ ]:
n = 1000
L = LowerTriangular(rand(n,n))
b = rand(n);

In [ ]:
@btime row_fs(L,b);

In [ ]:
@btime col_fs(L,b);

In [ ]:
@btime L\b;

---

### Exercise:

How do your `row_fs` and `col_fs` functions compare to `Julia`'s built-in `x = L\b`?

---

### Exercise:

Determine the total number of flops to solve an $n \times n$ triangular linear system using two different methods:

1. Directly count the flops in your nonrecursive code.
2. Count the flops using the recursive version of your code.

### Part 1

```julia
function col_fs(L, b)
    n = length(b)
    x = copy(b)
    for j = 1:n
        x[j] /= L[j,j]
        # Compute the reduced rhs
        for i = j+1:n
            x[i] -= L[i,j]*x[j]
        end
    end
    return x
end
```

1. 2 flops are required for the line `x[i] -= L[i,j]*x[j]`.
2. Then the `for i` loop does:
$$
\sum_{i=j+1}^n 2.
$$
3. The division `x[j] /= L[j,j]` is one flop.
4. Inside the `for j` loop is
$$
1 + \sum_{i=j+1}^n 2
$$
flops.
5. The `for j` loop does:
$$
\sum_{j=1}^n \left(1 + \sum_{i=j+1}^n 2\right).
$$

Suppose $n = 10$ and $j = 4$. Then

$$
\sum_{i=j+1}^n 2 = \sum_{i=5}^{10} 2 = (10 - 5 + 1) \cdot 2 = 12.
$$

In general, we have

$$
\sum_{i=j+1}^n 2 = (n - (j+1) + 1) \cdot 2 = 2(n - j).
$$

Thus, the total number of flops is
$$
\sum_{j=1}^n \left(1 + 2(n - j)\right) = \sum_{j=1}^n (1 + 2n) - 2 \sum_{j=1}^n j.
$$

A famous formula is that
$$
\sum_{j=1}^n j = 1 + 2 + \cdots + n = \frac{n(n+1)}{2}.
$$

Thus, the total flops are
\begin{eqnarray}
n(1 + 2n) - 2\frac{n(n+1)}{2} 
&=& n(1 + 2n) - n(n + 1)\\
&=& n + 2n^2 - n^2 - n\\
&=& n^2. \\
\end{eqnarray}

### Part 2

We can solve the first equation for $x_1$:

$$
x_1 = \frac{b_1}{l_{11}}.
$$

Substituting $x_1$ into the second equation gives us:

$$
\hat{L} \hat{x} = \hat{b} - x_1 \hat{l}.
$$

Let $T_n$ be the number of flops for forward substitution to solve an $n \times n$ linear system $Lx = b$.

First we note that $T_1 = 1$ because solving $l_{11} x_1 = b_1$ requires one division.

Now consider the $n \times n$ case. The recursive algorithm requires three steps:

1. calculating $x_1 = b_1/l_{11}$ is 1 flop;
2. computing $\bar{b} = \hat{b} - x_1 \hat{l}$ requires $2(n-1)$ flops since $\hat{b}$ and $\hat{l}$ are vectors of length $n-1$;
3. solving $\hat{L} \hat{x} = \bar{b}$ requires $T_{n-1}$ flops.

Thus,

$$
T_n = 1 + 2(n - 1) + T_{n-1}
$$

for $n > 1$.

Then
\begin{eqnarray}
T_n
&=& 2n - 1 + T_{n-1} \\
&=& 2n - 1 + 2(n-1) - 1 + T_{n-2} \\
&=& 2n - 1 + 2(n-1) - 1 + 2(n-2) - 1 + T_{n-3} \\
&\vdots& \\
&=& 2n - 1 + 2(n-1) - 1 + 2(n-2) - 1 + \cdots + T_{1}. \\
\end{eqnarray}

Finally,

\begin{eqnarray}
T_n 
&=& \big(2n + 2(n-1) + \cdots + 2 \cdot 2\big) + \big( - 1\cdot(n - 1) \big) + 1\\
&=& 2\sum_{i=2}^n i - (n - 1) + 1 \\
&=& 2\left( \frac{n(n+1)}{2} - 1 \right) - n + 2  \\
&=& n(n+1) - 2 - n + 2  \\
&=& n^2 + n - 2 - n + 2  \\
&=& n^2.
\end{eqnarray}

---

### Exercise:

Write a `Julia` function `col_fs!(b, L)` that **overwrites** the array `b` with the solution of the linear sytem $Lx = b$. Test the efficiency of your function.

In [ ]:
function col_fs!(b, L)
    n = length(b)
    for j = 1:n
        if L[j,j] == 0
            error("Error: Matrix L is singular.")
        end
        b[j] /= L[j,j]
        # Compute the reduced rhs
        for i = j+1:n
            b[i] -= L[i,j]*b[j]
        end
    end
    return b
end

In [ ]:
n = 10
L = LowerTriangular(rand(n,n))
b = rand(n);

In [ ]:
x = copy(b)
col_fs!(x, L)
L*x - b

In [ ]:
n = 1000
L = LowerTriangular(rand(n,n))
b = rand(n);

In [ ]:
@btime row_fs(L,b);

In [ ]:
@btime col_fs(L,b);

In [ ]:
@btime col_fs!($(copy(b)), L);

In [ ]:
@btime L\b;

In [ ]:
x = similar(b)
@btime ldiv!(x, L, b);

In [ ]:
?ldiv!

---